In [2]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import os
import numpy as np
import pandas as pd

In [ ]:
setup_logging()
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)

In [ ]:
#Check if the dataset has at least 1 resource of the required file type(s).

def check_type(dataset, file_types=[]):
    temp_dataset = Dataset.read_from_hdx(dataset)
    temp_dataset.separate_resources()
    if (len(temp_dataset.resources) > 0):
        if (len(file_types) > 0):
            if (not set(temp_dataset.get_filetypes()).isdisjoint(file_types)): 
                    return True
        else :
            return True
    return False

In [ ]:
# Check if the dataset is tagged with HXL tag, not provided by HXL

def check_organization(dataset):
    if dataset.get_organization()['title'] != 'Humanitarian Exchange Language(HXL)':
            return True
    return False

In [ ]:
# Download one dataset with certain type(s), read it into Dataframe, 
# add all headers, tags and dataset names to our DataFrame,
# and delete the dataset

def process_dataset(dataset, file_type, dataframe, download_path, index):
    global count
# Download one dataset and read it into a DataFrame 
    if (file_type == None):
        url, path = dataset_1.resources[0].download(download_path)
        pandas_dataset = pd.read_csv(path)
    else:
        if (file_type not in dataset.get_filetypes()):
            return 'Error: Required file type not in dataset OR dataset does not contain any resources.'
        try:
            url, path = dataset.resources[dataset.get_filetypes().index(file_type)].download(download_path)
            print('Resource URL %s downloaded to %s' % (url, path))
            pandas_dataset = pd.read_csv(path, encoding='latin-1')
        except:
            return 'Unknown error.'
     
    if "HXL" in os.path.basename(path) or "hxl" in os.path.basename(path):
        return
    dataset_df = pandas_dataset.head()
    
# Add headers and tags to our DataFrame
    if len(dataset_df) > 2:
        headers = list(dataset_df.columns.values)
        tags = list(dataset_df.iloc[0,:])
        for i in range(len(headers)):
            try:
                dic = {'Header': headers[i], 'Tag': tags[i], 'Dataset_name': os.path.basename(path), 'Index': index}
                headers_and_tags.loc[len(headers_and_tags)] = dic
            except:
                print("Error: different number of headers and tags")
        count += 1
    os.remove(path)
    print("File Removed!")
    return

In [ ]:
# Search for all datasets with HXL tags

datasets_HXL = Dataset.search_in_hdx('HXL')
len(datasets_HXL)

In [ ]:
# Create a DataFrame for all headers and tags

col_names = ['Header', 'Tag', 'Dataset_name', 'Index']
headers_and_tags= pd.DataFrame(columns = col_names)

In [ ]:
# Creat a temp DataFrame

col_names = ['Header', 'Tag', 'Dataset_name', 'Index']
temp = pd.DataFrame(columns = col_names)

In [ ]:
for i in range(1000):
    process_dataset(datasets_HXL[i], 'CSV', headers_and_tags, './datasets', count)


In [ ]:
datasets_HXL_2 = datasets_HXL[1000:1500]

In [ ]:
count = 665
for i in range(len(datasets_HXL_2)):
    process_dataset(datasets_HXL_2[i], 'CSV', temp, './datasets', count)

In [ ]:
headers_and_tags.to_excel("headerandtag.xlsx")